In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate,MaxPooling2D,BatchNormalization
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model,load_model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
import time
import random,pickle,cv2
from PIL import Image

Using TensorFlow backend.


In [12]:
files=['f/'+ x for x in os.listdir('../f/')]
labels=pickle.load(open('../fdata.pkl','rb'))
val_files=pickle.load(open('../fval_files.pkl','rb'))
random.shuffle(files)
train_files=[f for f in files if f not in val_files and len(labels[f])>0]
len(train_files)

18854

In [29]:

#Encoder
inputs = Input(shape=(41472,))
output2 = Dense(768, activation='relu')(inputs)
output2=keras.layers.Dropout(0.2)(output2)
output2 = Dense(300, activation='relu')(output2)
output2=keras.layers.Dropout(0.2)(output2)
output2 = Dense(100, activation='relu')(output2)
output2=keras.layers.Dropout(0.2)(output2)
predictions2 = Dense(2, activation='linear')(output2)



model = Model(inputs=inputs ,outputs=predictions2)
#model = Model(inputs=[input,embed_input], outputs=output)

In [4]:
graph = tf.get_default_graph()


In [5]:

#Generate training data
batch_size = 20

def train_gen(batch_size,model1):
    while True:
        i=random.randint(0,len(train_files)-batch_size-1)
        names=train_files[i:i+batch_size]
        X_batch, Y_batch=generator(names,model1)
        yield (X_batch, Y_batch)
        
def val_gen(model1):
    names=val_files
    
    X_batch, Y_batch=generator(names,model1)
    while True:
            yield (X_batch,Y_batch)
def generator(names,model1):
    X_batch=[]
    Y_batch=[]
    embed = []
    for filename in names:
        X_batch.append(img_to_array(load_img('../'+filename))/255)
        d=[]
        if(len(labels[filename])>0):    
            d+=[1,labels[filename][0]/299.0,labels[filename][1]/299.0]
        if(len(d)<3):
            d=[0,0,0]
        Y_batch.append(d)    
    X_batch=np.array(X_batch)
    embed=create_embedding(model1,X_batch)
    Y_batch=np.array(Y_batch)
    #embed=create_inception_embedding(X_batch)
    return embed, Y_batch


In [6]:
def f2_score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def loss(y_true, y_pred):
    return keras.losses.binary_crossentropy(y_true,y_pred)

def loss2(y_true, y_pred):
    A=y_pred[:,0]*y_true[:,0]
    B=y_pred[:,1]*y_true[:,0]
    a=K.sum(K.square(A-y_true[:,1]))+K.sum(K.square(B-y_true[:,2]))
    return a

In [7]:
model1=load_model('model3.hdf5', custom_objects={'loss': loss,'f2_score':f2_score})
def create_embedding(model1,x):
    global graph
    with graph.as_default():
            get_3rd_layer_output = K.function([model1.layers[0].input],
                                  [model1.layers[-8].output])
            layer_output = get_3rd_layer_output([x])[0]
    return layer_output

def get_3rd_layer_output(model1,x):
    global graph
    with graph.as_default():
            get_3rd_layer_output = K.function([model1.layers[0].input],
                                  [model1.layers[-1].output])
            layer_output = get_3rd_layer_output([x])[0]
    return layer_output


In [32]:
adam=keras.optimizers.Adam(lr=0.00001)
tensorboard = TensorBoard(log_dir="./output")
model.compile(optimizer=adam, loss=loss2)

In [33]:
def exp_decay(step):
   initial_lrate = 0.0001
   final_lrate=0.00001
   lrate = (initial_lrate-final_lrate)*step/150
   print(initial_lrate-lrate)
   return initial_lrate-lrate

lrate = keras.callbacks.LearningRateScheduler(exp_decay)

In [ ]:


#Train model      

model.fit_generator(train_gen(batch_size,model1), epochs=150,steps_per_epoch=40,validation_data=val_gen(model1),validation_steps=1, max_queue_size=15)

Epoch 1/150
40/40 [==============================] - 34s 860ms/step - loss: 0.8578 - val_loss: 0.9412
Epoch 2/150
40/40 [==============================] - 36s 907ms/step - loss: 0.8802 - val_loss: 0.9766
Epoch 3/150
40/40 [==============================] - 38s 945ms/step - loss: 0.7291 - val_loss: 0.9578
Epoch 4/150
40/40 [==============================] - 35s 869ms/step - loss: 0.7772 - val_loss: 0.9487
Epoch 5/150
40/40 [==============================] - 35s 877ms/step - loss: 1.7272 - val_loss: 0.9346
Epoch 6/150
40/40 [==============================] - 34s 843ms/step - loss: 0.8239 - val_loss: 0.9146
Epoch 7/150
40/40 [==============================] - 37s 934ms/step - loss: 0.7481 - val_loss: 0.9167
Epoch 8/150
40/40 [==============================] - 36s 904ms/step - loss: 0.8239 - val_loss: 0.9374
Epoch 9/150
40/40 [==============================] - 38s 941ms/step - loss: 0.7943 - val_loss: 0.9501
Epoch 10/150
40/40 [==============================] - 36s 890ms/step - loss: 0.880

In [ ]:
model.save('model3regress.hdf5')

In [34]:
model.load_weights('model3regress.hdf5')

In [ ]:
x,y=next(train_gen(1,model1))
Y=model.predict(create_embedding(model1,x))
Y1=model1.predict(x)
print(Y.shape,Y1.shape)
if Y1[0]>0.5:
    print(Y[0])
    cv2.circle(x[0],tuple(map(int,Y[0]*299)),1,(0,255,0),5)
plt.imshow(x[0])

In [ ]:
def plotRec(vec,Y):
    index=np.argsort(Y,axis=0)[::-1]
    print(Y[index])
    x=vec[index]
    return x

In [19]:
def manipulateImage(img):
    height=299*(1+(img.shape[0]//299))
    width=299*(1+(img.shape[1]//299))
    h=height-img.shape[0]
    w=width-img.shape[1]
    print( h//2, h-(h//2), w//2, w-(w//2))
    image = cv2.copyMakeBorder( img, h//2, h-(h//2), w//2, w-(w//2), cv2.BORDER_CONSTANT)
    #plt.imshow(img)
    vec=[]
    for x in range(width//299):
        for y in range(height//299):
            vec.append(image[299*y:299*(y+1),299*x:299*(x+1)])
    return np.array(vec)

vec=manipulateImage(img_to_array(load_img('../CV-Assignment-Dataset/DSC01453.JPG'))/255)
print(vec.shape)

Y=model.predict(create_embedding(model1,vec))
model.predict(vec)
vec=plotRec(vec,model.predict(vec))
# import time
f=plt.figure(figsize=(10,25))
i=1
for x in vec[:5]:
    print(x.shape)
    f.add_subplot(5,1,i)
    plt.imshow(x[0])
    i+=1

    #     time.sleep(3)
    

93 93 139 140
(294, 299, 299, 3)


ResourceExhaustedError: OOM when allocating tensor with shape[294,32,299,299] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_12_1/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_12_1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_12_1/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: reshape_2_1/Reshape/_20077 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_401_reshape_2_1/Reshape", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list o

In [ ]:
print(495e-5)